Dimensionality Reduction Work

Approaches:
- Use Financial Journal, combined with Household size


To Do:
- https://www.kaggle.com/code/tilii7/dimensionality-reduction-pca-tsne
- https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf
- https://distill.pub/2016/misread-tsne/

In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import altair as alt

In [2]:
participants_path = Path(r"/content/drive/MyDrive/Datasets/VAST_Challenge_2022/Attributes/Participants.csv").resolve()
financial_journal_path = Path(r"/content/drive/MyDrive/Datasets/VAST_Challenge_2022/Journals/FinancialJournal.csv").resolve()
df_participants = pd.read_csv(participants_path)
df_financials = pd.read_csv(financial_journal_path)

In [3]:
PARSING_MAPS = {
    'travelStartTime':"%Y-%m-%d %H:%M:%S",
    'travelEndTime':"%Y-%m-%d %H:%M:%S",
    'timestamp' : "%Y-%m-%d %H:%M:%S"
}

def processDateTime(df,cols):
    """
    :params df: dataframe
    :params alias:columns alias thats in COL_MAP
    """
    for col in cols:
        df[col] = pd.to_datetime(df[col],format=PARSING_MAPS[col])
    return df

def addTimeColumns(df,cols,main):
  for col in cols:
    df[col] = getattr(df[main].dt,col)
  return df

def monthlyIncomeExpenses(df):
  df_dict = {}
  cols = ['participantId']
  for key,grp in df.groupby('participantId'):
      categoryGroupBy = grp.groupby([pd.Grouper(key = 'timestamp',freq='M'),'category'])['amount'].sum()
      categoryGroupBy = categoryGroupBy.reset_index()

      categoryGroupBy.loc[categoryGroupBy['amount'] > 0,'category'] = 'Income'
      categoryGroupBy.loc[categoryGroupBy['amount'] < 0,'category'] = 'Expense'

      incomeExpenseGroupBy = categoryGroupBy.groupby([pd.Grouper(key = 'timestamp',freq = 'M'),'category'])['amount'].sum()
      incomeExpenseGroupBy = incomeExpenseGroupBy.reset_index()
      incomeExpenseGroupBy['year'] = incomeExpenseGroupBy['timestamp'].dt.year
      incomeExpenseGroupBy['month'] = incomeExpenseGroupBy['timestamp'].dt.month

      incomeExpenseGroupBy = incomeExpenseGroupBy[['category','amount','year','month']]\
                            .pivot(columns = 'category',values = 'amount',index = ['year','month'])

      df_dict[key] = pd.Series(np.hstack((incomeExpenseGroupBy['Expense'],incomeExpenseGroupBy['Income'])))

  expense_cols = ['_'.join(['Expense',str(i),str(j)]) for i,j in \
                  (zip(incomeExpenseGroupBy.reset_index()['year'],incomeExpenseGroupBy.reset_index()['month']))]
  income_cols = ['_'.join(['Income',str(i),str(j)]) for i,j in \
                 (zip(incomeExpenseGroupBy.reset_index()['year'],incomeExpenseGroupBy.reset_index()['month']))]
  cols.extend(expense_cols)
  cols.extend(income_cols)

  df = pd.concat(df_dict,axis = 1).transpose()
  df.reset_index(inplace = True)
  df.columns = cols
  return df

In [4]:
df_financials = processDateTime(df_financials,['timestamp'])
df_financials = addTimeColumns(df_financials,['year','month','day'],'timestamp')
df_result = monthlyIncomeExpenses(df_financials)
#drop na
df_result = df_result.dropna(axis = 0)

In [29]:
col_dict = {}
for col in df_result.columns.tolist():
  col_dict[col] = df_result.loc[df_result[col].isna()].shape[0]

In [32]:
df_result

,participantId,Expense_2022_3,Expense_2022_4,Expense_2022_5,Expense_2022_6,Expense_2022_7,Expense_2022_8,Expense_2022_9,Expense_2022_10,Expense_2022_11,...,Income_2022_8,Income_2022_9,Income_2022_10,Income_2022_11,Income_2022_12,Income_2023_1,Income_2023_2,Income_2023_3,Income_2023_4,Income_2023_5
0,0,-1803.043427,-1078.279044,-1240.618894,-1315.647934,-1932.750901,-1168.974312,-1143.759008,-884.744162,-1231.510507,...,9459.440064,9048.160061,8636.880059,9048.160061,9048.160061,9048.160061,8225.600056,9459.440064,8225.600056,7403.040050
1,1,-2522.843157,-1364.961681,-1209.680147,-1529.355645,-1368.730604,-1301.937676,-1235.970307,-1239.735119,-1478.765318,...,8312.461423,7951.050057,7589.638691,7951.050057,7951.050057,7951.050057,7228.227325,8312.461423,7228.227325,6505.404592
2,2,-1860.520294,-1200.120522,-1183.578950,-1202.767051,-1245.152416,-1301.563289,-1135.375980,-1275.081532,-1105.825139,...,7285.149162,6968.403546,6651.657931,6968.403546,6968.403546,6968.403546,6334.912315,7285.149162,6334.912315,5701.421083
3,3,-1931.792313,-1195.355238,-1663.021624,-1198.765871,-1144.680563,-1602.361630,-1178.308993,-1285.489830,-1166.077677,...,7041.234434,6735.093806,6428.953179,6735.093806,6735.093806,6735.093806,6122.812551,7041.234434,6122.812551,5510.531296
4,4,-9056.661441,-1948.813690,-1864.045474,-1662.446157,-1710.513099,-1608.675426,-1721.997659,-1826.201137,-1683.411613,...,9129.007681,8732.094304,8335.180927,8732.094304,8732.094304,8732.094304,7938.267549,9129.007681,7938.267549,7144.440794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,1006,-2236.410066,-1388.886993,-1303.270588,-1151.944369,-1272.094207,-1246.057191,-1144.875532,-1164.754694,-1170.118840,...,1840.000000,1760.000000,1680.000000,1760.000000,1760.000000,1760.000000,1600.000000,1840.000000,1600.000000,1440.000000
1007,1007,-2850.303449,-1495.890342,-1492.665709,-1396.318139,-1674.261763,-1427.864909,-1528.363537,-1727.965687,-1688.382786,...,1840.000000,1760.000000,1680.000000,1760.000000,1760.000000,1760.000000,1600.000000,1840.000000,1600.000000,1440.000000
1008,1008,-1852.109087,-1171.279313,-1333.390588,-989.424348,-846.839250,-820.729606,-802.304225,-863.355012,-847.848229,...,3778.148881,3613.881538,3449.614196,3613.881538,3613.881538,3613.881538,3285.346853,3778.148881,3285.346853,2956.812168
1009,1009,-2230.784099,-1579.051700,-1413.483960,-1395.092872,-1436.051791,-1620.433695,-1411.968501,-1479.244578,-1400.507684,...,1859.166667,1778.333333,1697.500000,1778.333333,1778.333333,1778.333333,1616.666667,1859.166667,1616.666667,1455.000000


In [36]:
df_result.loc[df_result['participantId'] == 926]

,participantId,Expense_2022_3,Expense_2022_4,Expense_2022_5,Expense_2022_6,Expense_2022_7,Expense_2022_8,Expense_2022_9,Expense_2022_10,Expense_2022_11,...,Income_2022_8,Income_2022_9,Income_2022_10,Income_2022_11,Income_2022_12,Income_2023_1,Income_2023_2,Income_2023_3,Income_2023_4,Income_2023_5
926,926,-1195.66034,-558.567954,-563.487929,-555.487929,-560.919778,-558.759728,-556.895224,-564.21613,-557.839753,...,2370.795062,2267.717016,2164.63897,2267.717016,2267.717016,2267.717016,2061.560923,2370.795062,2061.560923,1855.404831


In [7]:
cols_expenses = [i for i in df_result.columns if "Expense" in i]
col_income = [i for i in df_result.columns if "Income" in i]

In [20]:
df_average_expenses = df_result[cols_expenses].mean(axis = 1)
df_average_income = df_result[col_income].mean(axis = 1)

In [24]:
df_expense_income = pd.concat((pd.DataFrame({'pId':range(0,1011)}),df_average_expenses,df_average_income),axis = 1)

In [26]:
df_expense_income.columns = ['pId','Average Expenses','Average Income']

In [28]:
df_expense_income.to_json('/content/drive/MyDrive/Datasets/VAST_Challenge_2022/Expense_Income.json')

In [ ]:
scaler = StandardScaler()
df_tsne = df_participants[['participantId','educationLevel','householdSize','age']].merge(df_result,how = 'left',on = 'participantId')
df_tsne = df_tsne.dropna(axis = 0)
df_tsne[[i for i in df_tsne.columns.tolist() if i not in  ["participantId",'educationLevel']]] = scaler.fit_transform(df_tsne[[i for i in df_tsne.columns.tolist() if i not in  ["participantId",'educationLevel']]])

In [ ]:
tsne_obj = TSNE(n_components = 2,n_iter = 5000,perplexity = 20)
tsne_res = tsne_obj.fit_transform(df_tsne[[i for i in df_tsne.columns.tolist() if i not in  ["participantId",'educationLevel']]])

In [ ]:
df_tsne_res = pd.DataFrame(data = {'participantId':df_tsne['participantId'],\
                                   'educationLevel':df_tsne['educationLevel'],\
                                   'X1':tsne_res[:,0],'X2':tsne_res[:,1]})

In [ ]:
viz = alt.Chart(df_tsne_res)\
.mark_circle()\
.encode(
    x = 'X1',
    y = 'X2',
    color = 'educationLevel')\
.properties(
    width = 1000,
    height = 1000
)
viz

In [ ]:
df_participants